In [1]:
# === IMPORT LIBRARY ===
import os
import pandas as pd
import psycopg2
from vanna.ollama import Ollama
from vanna.chromadb import ChromaDB_VectorStore

# === SETUP PATH ===
cdir = os.getcwd()
chroma = os.path.join(cdir, "database")
training_csv = os.path.join(cdir, "training", "dataset.csv")

# === CLASS VNANNA: COMBINE OLLAMA + CHROMADB ===
class Vanna(ChromaDB_VectorStore, Ollama):
    def __init__(self, config=None) -> None:
        ChromaDB_VectorStore.__init__(self, config=config)
        Ollama.__init__(self, config=config)

# === INSTANSIASI MODEL OLLAMA + CHROMA ===
print("🚀 Inisialisasi Vanna dengan model dan ChromaDB...")
vn = Vanna(
    config={
        "model": "qwen2.5:3b",
        "path": chroma,
    }
)
print("✅ Vanna siap!")

# === GANTI FUNGSI RUN_SQL DENGAN VERSI MANUAL (psycopg2) ===
def manual_run_sql(sql: str) -> pd.DataFrame:
    try:
        conn = psycopg2.connect(
            host="localhost",
            port=5433,
            user="postgres",
            password="KuRn1@26",
            dbname="data_telkomsel"
        )
        df = pd.read_sql_query(sql, conn)
        conn.close()
        return df
    except Exception as e:
        print("❌ Error pada manual_run_sql:", e)
        return pd.DataFrame()

vn.run_sql = manual_run_sql  # Override default run_sql()

# === TRAINING DARI DDL (schema/tabel di PostgreSQL) + COMMENT ===
try:
    print("🔍 Mengambil daftar tabel dari PostgreSQL...")
    ddls = vn.run_sql("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")

    if ddls.empty:
        print("⚠️ Tidak ada tabel ditemukan.")
    else:
        print(f"📋 Ditemukan {len(ddls)} tabel:", ddls["table_name"].tolist())

        for table_name in ddls["table_name"]:
            print(f"📦 Memproses tabel: {table_name}...")

            # Ambil DDL utama
            ddl_query = f"""
            SELECT 'CREATE TABLE {table_name} (\n' ||
                   string_agg('    ' || a.attname || ' ' || format_type(a.atttypid, a.atttypmod), ',\n') ||
                   '\n);' AS ddl
            FROM pg_class c
            JOIN pg_attribute a ON a.attrelid = c.oid
            WHERE c.relname = '{table_name}' AND a.attnum > 0 AND NOT a.attisdropped;
            """
            ddl_result = vn.run_sql(ddl_query)

            # Ambil komentar kolom
            col_comment_query = f"""
            SELECT a.attname AS column_name, d.description AS comment
            FROM pg_class c
            JOIN pg_attribute a ON a.attrelid = c.oid
            LEFT JOIN pg_description d ON d.objoid = a.attrelid AND d.objsubid = a.attnum
            WHERE c.relname = '{table_name}' AND a.attnum > 0 AND NOT a.attisdropped;
            """
            col_comments = vn.run_sql(col_comment_query)

            # Ambil komentar tabel
            tbl_comment_query = f"""
            SELECT d.description AS comment
            FROM pg_class c
            LEFT JOIN pg_description d ON d.objoid = c.oid AND d.objsubid = 0
            WHERE c.relname = '{table_name}';
            """
            tbl_comment = vn.run_sql(tbl_comment_query)

            if not ddl_result.empty:
                ddl_base = ddl_result.iloc[0, 0]
                comment_block = f"-- DDL for table `{table_name}`\n"

                if not tbl_comment.empty and pd.notnull(tbl_comment.iloc[0, 0]):
                    comment_block += f"-- Table comment: {tbl_comment.iloc[0, 0]}\n"

                if not col_comments.empty:
                    for _, row in col_comments.iterrows():
                        if pd.notnull(row["comment"]):
                            comment_block += f"-- Column `{row['column_name']}`: {row['comment']}\n"

                final_ddl = comment_block + ddl_base
                vn.train(ddl=final_ddl)
                print(f"✅ Training DDL + COMMENT sukses untuk tabel: {table_name}")
            else:
                print(f"⚠️ DDL kosong untuk tabel: {table_name}")

except Exception as e:
    print("❌ Gagal saat training DDL + COMMENT:", e)

# === TRAINING DARI CSV Q&A ===
try:
    print("📥 Membaca file training CSV...")
    df = pd.read_csv(training_csv, delimiter=";", encoding="utf-8")

    if df.empty or "question" not in df.columns or "answer" not in df.columns:
        raise ValueError("CSV kosong atau kolom 'question' / 'answer' tidak ditemukan.")

    print(f"✅ CSV dibaca, {len(df)} baris ditemukan.")

    for index, row in df.iterrows():
        question = str(row["question"]).strip()
        answer = str(row["answer"]).strip() + ";"
        try:
            vn.train(question=question, sql=answer)
            print(f"✅ Q{index+1}: Training berhasil — \"{question}\"")
        except Exception as e:
            print(f"❌ Q{index+1}: Gagal training — \"{question}\" | Error: {e}")
except Exception as e:
    print("❌ Gagal membaca atau memproses CSV:", e) 

🚀 Inisialisasi Vanna dengan model dan ChromaDB...


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


✅ Vanna siap!
🔍 Mengambil daftar tabel dari PostgreSQL...


C:\Users\rafi\AppData\Local\Temp\ipykernel_25416\1504409858.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


📋 Ditemukan 1 tabel: ['data_performance']
📦 Memproses tabel: data_performance...
Adding ddl: -- DDL for table `data_performance`
-- Table comment: Tabel "data_performance" menyimpan data transaksi harian dari berbagai channel penjualan Telkomsel. Tabel ini digunakan untuk menganalisis performa penjualan berdasarkan dimensi waktu (`trx_date`), wilayah (`region`, `area`), produk (`product_commercial_name`), kampanye pemasaran (`campaign_initiatives`), channel distribusi (`channel_name`), dan metode pembayaran (`payment_method`). Data mencakup metrik utama seperti jumlah transaksi (`trx`), total pendapatan (`revenue`), dan jumlah pengguna unik (`taker_uniq_all`). Tabel ini penting untuk pelaporan, evaluasi kampanye, dan pengambilan keputusan berbasis data. Query terhadap tabel ini akan menggunakan sintaks PostgreSQL.
-- Column `trx_date`: Tanggal terjadinya transaksi.
-- Column `groups`: Segmentasi pelanggan berdasarkan tahap campaign seperti Conversion atau Retention
-- Column `area`: Ar

...

✅ Q397: Training berhasil — "Tampilkan 5 area dengan pengguna unik paling sedikit di bulan Januari 2025"
✅ Q398: Training berhasil — "Identifikasi channel dengan selisih revenue tertinggi dibanding bulan sebelumnya (Februari ke Maret 2025)"
✅ Q399: Training berhasil — "Cari tahu kombinasi channel dan metode pembayaran yang menghasilkan transaksi terbanyak pada Desember 2024"
✅ Q400: Training berhasil — "Tampilkan semua campaign yang aktif di region 08.Jateng selama Februari 2025"


Insert of existing embedding ID: 3babe54a-a467-558a-ae5d-f8378dd954c4-sql
Add of existing embedding ID: d3835775-f4cf-50b2-b451-7bad620a77e0-sql
Insert of existing embedding ID: d3835775-f4cf-50b2-b451-7bad620a77e0-sql
Add of existing embedding ID: 316d87eb-941b-541e-96f3-4caff698d845-sql
Insert of existing embedding ID: 316d87eb-941b-541e-96f3-4caff698d845-sql
Add of existing embedding ID: b802b8ec-7689-5cc2-85de-bad4fd81437f-sql
Insert of existing embedding ID: b802b8ec-7689-5cc2-85de-bad4fd81437f-sql
Add of existing embedding ID: 79cabea9-a0a5-5047-822d-7ac7ea19ea96-sql


✅ Q401: Training berhasil — "Tampilkan rata-rata transaksi per pengguna unik untuk tiap area di Januari 2025"
✅ Q402: Training berhasil — "Tampilkan distribusi transaksi untuk produk tertentu di setiap region pada bulan Maret 2025"
✅ Q403: Training berhasil — "Cari tahu area mana saja yang hanya menggunakan satu metode pembayaran selama Desember 2024"
✅ Q404: Training berhasil — "Tampilkan 3 channel teratas dengan rasio revenue per transaksi tertinggi di Januari 2025"


Insert of existing embedding ID: 79cabea9-a0a5-5047-822d-7ac7ea19ea96-sql
Add of existing embedding ID: c6b5d07a-ea21-5425-a8bd-741eccf3b0ed-sql
Insert of existing embedding ID: c6b5d07a-ea21-5425-a8bd-741eccf3b0ed-sql
Add of existing embedding ID: a0c8ca19-757b-560f-999e-bb33c80132b6-sql
Insert of existing embedding ID: a0c8ca19-757b-560f-999e-bb33c80132b6-sql


✅ Q405: Training berhasil — "Tampilkan 5 kombinasi region dan produk dengan revenue terbesar di Februari 2025"
✅ Q406: Training berhasil — "Sebutkan seluruh kombinasi area dan channel yang menghasilkan revenue di bawah 10 juta pada Maret 2025"
✅ Q407: Training berhasil — "Identifikasi campaign yang menghasilkan revenue di atas rata-rata di bulan Januari 2025"
✅ Q408: Training berhasil — "Tunjukkan region dengan revenue tertinggi selama bulan Desember 2024"
✅ Q409: Training berhasil — "Sebutkan tiga produk dengan jumlah transaksi terbanyak di Januari 2025"
✅ Q410: Training berhasil — "Saluran penjualan apa yang memiliki taker unik tertinggi sepanjang Maret 2025?"
✅ Q411: Training berhasil — "Apa saja metode pembayaran yang digunakan di bulan Februari 2025?"
✅ Q412: Training berhasil — "Tampilkan 5 area dengan revenue terendah selama bulan Desember 2024"
✅ Q413: Training berhasil — "Di bulan Januari 2025, produk apa yang menghasilkan revenue paling besar?"
✅ Q414: Training berhasil — "Ma